# Importing the required libraries 

In [63]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import glob
import os

In [64]:
data = pd.read_csv('youtube20.csv', encoding='ISO-8859-1')
data.head()
data.shape
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
Discern Scole Score    20 non-null int64
Upvotes                20 non-null int64
Downvotes              20 non-null int64
Views                  20 non-null int64
URL                    20 non-null object
Transcript             17 non-null object
Comment sentiment      7 non-null float64
Informative            20 non-null int64
dtypes: float64(1), int64(5), object(2)
memory usage: 1.3+ KB


# Function to remove stop words,pre-process text & tokenize

In [65]:
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from nltk.corpus import stopwords 
# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
stop_words = set(stopwords.words('english'))

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [66]:
# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return str(text).strip().lower()

In [67]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,5))

In [68]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

# Split the data into train and test set

In [69]:
from sklearn.model_selection import train_test_split

X = data['Transcript'] # the features we want to analyze
ylabels = data['Informative'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)

# Cleaning, Vectorizer and Logistic regression classifier

In [70]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('cleaner', <__main__.predictors object at 0x000001E2B1B0AA58>), ('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
      ...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

# Running the pipeline and predicting using Logistic regression

In [71]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))

Logistic Regression Accuracy: 0.6666666666666666
Logistic Regression Precision: 0.0
Logistic Regression Recall: 0.0
